In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Genie: From a Single Image to an Interactive World

**Notebook 5 of 6 — World Action Models Series | Vizuara**

What if you could sketch a game level on a napkin and then *play* it? What if a single photograph of a mountain trail could become a world you walk through? This is exactly what Genie does — it generates fully interactive environments from a single image, and it learns to do this *without ever being told what the actions are*.

By the end of this notebook, you will build a simplified Genie-style model that **learns actions from unlabeled video** and **generates interactive frame sequences** — all from scratch in PyTorch.

**Estimated time: ~40 minutes**

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/world-action-models/practice/5/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# --- Device Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# --- Reproducibility ---
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

%matplotlib inline
print("Setup complete!")

## 1. Why Does This Matter?

Let us start with a thought experiment. Imagine you hand a child a crayon drawing of a simple platformer game — a stick figure, some platforms, maybe a flag at the top. Now imagine the child could *step inside* that drawing and actually play it. Jump on the platforms. Run left and right. Reach the flag.

This is not science fiction. This is **Genie**.

Genie, developed by Google DeepMind, takes a single image and generates a fully interactive world from it. The user can take actions — move left, move right, jump — and the world responds, frame by frame, as if it were a real game.

But here is the truly remarkable part: **Genie learns what "actions" are without ever being told.** No one labels the training data with "this is a jump" or "this is a move right." The model watches thousands of unlabeled videos and *discovers* that there are consistent controls that transform one frame into the next.

This brings us to an important question: **How does a model learn what actions exist just by watching?**

## 2. Building Intuition

### The Problem with Previous World Models

In our earlier notebooks, world models like Ha and Schmidhuber's "World Models" required **explicit action labels** during training. Someone had to tell the model: "Between frame $t$ and frame $t+1$, the agent pressed the 'right' button."

This is expensive. It means you need either:
1. A human to label every frame transition with the action taken, or
2. A simulator that provides action labels automatically.

What if we want to learn from the billions of hours of unlabeled video on the internet? YouTube gameplay videos, nature documentaries, dashcam footage — none of these come with action labels. **Genie solves this.**

### The Child Watching a Video Game

Let us build intuition with an analogy that captures the core idea.

Imagine a child sitting behind someone playing a video game. The child can see the screen perfectly, but **cannot see the controller at all**. The child watches for hours.

After enough watching, something remarkable happens. The child starts to understand:
- "Something causes the character to move right"
- "Something causes the character to jump"
- "Something causes the character to stop"

The child has **inferred the existence of actions** without ever seeing the controller. They do not know that these actions correspond to specific buttons — they just know that *some latent cause* consistently produces certain visual effects.

This is precisely what Genie's **Latent Action Model** does. It watches pairs of consecutive frames and learns: "Something happened between these two frames, and I will represent that something as a discrete code."

### Three Components: Eyes, Instinct, and Imagination

Genie has three core components. Let us map them to something intuitive:

| Component | Analogy | What It Does |
|-----------|---------|-------------|
| **Video Tokenizer** | Eyes | Compresses each frame into a compact grid of discrete tokens |
| **Latent Action Model** | Instinct | Looks at two consecutive frames and infers what action was taken |
| **Dynamics Model** | Imagination | Given a frame and an action, predicts what happens next |

The pipeline works like this:

1. **Eyes (Tokenizer):** Take a raw image frame and convert it into a compact, discrete representation — a grid of tokens from a learned vocabulary.
2. **Instinct (Latent Action Model):** Given two consecutive tokenized frames, infer the latent action that caused the transition. No supervision needed!
3. **Imagination (Dynamics Model):** Given the current tokenized frame and a chosen action, predict the next tokenized frame.

At inference time, a user provides a single starting image and a sequence of actions. The dynamics model unrolls the world forward, frame by frame, generating an interactive experience.

### 🤔 Think About This

Before we move to the mathematics, pause and consider:

> **How would YOU figure out what "actions" exist just by watching game footage?**
>
> You see thousands of frame pairs. Some pairs show the character moving right. Some show a jump. Some show no movement at all.
>
> What patterns would you look for? How would you group these transitions into a small number of categories?
>
> *Take a moment to think about this. The answer is at the heart of Genie.*

## 3. The Mathematics

Now let us formalize the three components. We will go through each one, show the equation, and then explain exactly what it means computationally.

### 3.1 Video Tokenizer

The tokenizer converts a continuous image into a discrete representation using **Vector Quantized Variational Autoencoder (VQ-VAE)**.

$$t_i = \text{VQ}(\text{Encoder}(x_i))$$

where $x_i \in \mathbb{R}^{H \times W \times 3}$ is the raw image frame and $t_i$ is a grid of discrete token indices.

**What this means computationally:**

1. The encoder (a CNN) takes the raw frame $x_i$ and produces a continuous feature map $z_i \in \mathbb{R}^{h \times w \times D}$, where $h, w$ are the spatial dimensions after downsampling and $D$ is the embedding dimension.
2. We maintain a **codebook** $\mathcal{C} = \{e_1, e_2, \ldots, e_K\}$ of $K$ learnable vectors, each of dimension $D$.
3. For each spatial position in $z_i$, we find the **nearest codebook vector**: $t_{i,j} = \arg\min_k \|z_{i,j} - e_k\|_2$.
4. The quantized output replaces each continuous vector with its nearest codebook entry.

**Numerical example:** Suppose $D = 4$ and $K = 3$. Our codebook has three vectors:
- $e_1 = [1.0, 0.0, 0.0, 0.0]$
- $e_2 = [0.0, 1.0, 0.0, 0.0]$
- $e_3 = [0.0, 0.0, 1.0, 0.0]$

If the encoder outputs $z = [0.9, 0.1, 0.05, 0.0]$ at some spatial position, we compute:
- $\|z - e_1\|_2 = \sqrt{0.01 + 0.01 + 0.0025 + 0} = 0.15$
- $\|z - e_2\|_2 = \sqrt{0.81 + 0.81 + 0.0025 + 0} = 1.27$
- $\|z - e_3\|_2 = \sqrt{0.81 + 0.01 + 0.9025 + 0} = 1.31$

The nearest codebook vector is $e_1$, so the token index at this position is $t = 1$. This is exactly what we want — the continuous encoder output has been "snapped" to the closest entry in our discrete vocabulary.

### 3.2 Latent Action Model (LAM)

The Latent Action Model infers **what action caused the transition** between two consecutive frames:

$$a_t = \text{LAM}(t_t, t_{t+1})$$

where $t_t$ and $t_{t+1}$ are the tokenized representations of consecutive frames, and $a_t \in \{1, 2, \ldots, N_a\}$ is a discrete latent action.

**What this means computationally:**

1. Encode both tokenized frames (or their continuous embeddings) through a small network.
2. Compute a difference or concatenation that captures "what changed."
3. Output a categorical distribution over $N_a$ possible latent actions.
4. Sample (or take argmax of) this distribution to get the discrete action $a_t$.

The key insight: **no one tells the model what these actions mean.** The model discovers that the space of frame transitions can be clustered into $N_a$ distinct categories. If the training data contains platformer games, the model might discover actions that correspond to "left," "right," "jump," and "idle" — but these labels are never provided.

**Numerical example:** Suppose $N_a = 4$ and the LAM sees two frames where a character moved to the right. The LAM outputs logits $[0.1, 2.5, -0.3, 0.2]$. Applying softmax:

$$p = \text{softmax}([0.1, 2.5, -0.3, 0.2]) = [0.07, 0.78, 0.05, 0.08]$$

The model assigns 78% probability to action index 1. Over time, all "move right" transitions get mapped to this same index. This is exactly what we want — the model has discovered an action category from unlabeled data.

### 3.3 Dynamics Model

The dynamics model predicts the next tokenized frame given the current frame and an action:

$$\hat{t}_{t+1} = \text{Dynamics}(t_t, a_t)$$

**What this means computationally:**

1. Take the current tokenized frame $t_t$ and embed each token using the codebook.
2. Embed the latent action $a_t$ into a vector.
3. Combine the frame embedding and action embedding (e.g., via concatenation or addition).
4. Pass through a neural network (CNN or Transformer) that predicts the token indices for the next frame.

This is the component that enables **interactivity**. At inference time, you provide:
- A starting frame $t_0$
- A sequence of actions $a_0, a_1, a_2, \ldots$

And the dynamics model generates: $\hat{t}_1 = \text{Dynamics}(t_0, a_0)$, then $\hat{t}_2 = \text{Dynamics}(\hat{t}_1, a_1)$, and so on — creating a controllable, interactive world.

### 3.4 Training Objective

The three components are trained jointly. The total loss has three terms:

$$\mathcal{L} = \underbrace{\mathcal{L}_{\text{recon}}}_{\text{Tokenizer}} + \beta \underbrace{\mathcal{L}_{\text{commit}}}_{\text{VQ Regularizer}} + \underbrace{\mathcal{L}_{\text{dynamics}}}_{\text{Next-frame prediction}}$$

- **Reconstruction loss** $\mathcal{L}_{\text{recon}} = \|x - \hat{x}\|_2^2$: The tokenizer should faithfully reconstruct frames.
- **Commitment loss** $\mathcal{L}_{\text{commit}} = \|z - \text{sg}(e)\|_2^2$: The encoder outputs should stay close to codebook entries ($\text{sg}$ means stop-gradient).
- **Dynamics loss** $\mathcal{L}_{\text{dynamics}} = \text{CrossEntropy}(\hat{t}_{t+1}, t_{t+1})$: The predicted next tokens should match the actual next tokens.

Let us now build all of this from scratch.

## 4. Let Us Build It

We will build a simplified Genie using a procedurally generated sprite dataset. Our sprite (a colored square) moves on a grid with 4 possible actions: up, down, left, right. Crucially, we will **only store the frames, not the actions** — the model must discover the actions on its own.

### 4.1 Create the Sprite Dataset

In [ ]:
# --- Constants for our sprite world ---
GRID_SIZE = 16         # 16x16 pixel frames
SPRITE_SIZE = 4        # 4x4 pixel sprite
NUM_ACTIONS = 4        # up, down, left, right
SEQ_LENGTH = 8         # frames per sequence
NUM_SEQUENCES = 3000   # total training sequences

# Action definitions (hidden from the model!)
# 0=up, 1=down, 2=left, 3=right
ACTION_DELTAS = {
    0: (-1, 0),   # up
    1: (1, 0),    # down
    2: (0, -1),   # left
    3: (0, 1),    # right
}

def generate_sprite_sequence(seq_length, grid_size, sprite_size):
    """Generate a sequence of frames with a moving sprite.

    Returns frames only — actions are NOT returned.
    Also returns actions separately for evaluation only.
    """
    frames = []
    actions_taken = []

    # Random starting position
    max_pos = grid_size - sprite_size
    row = np.random.randint(0, max_pos + 1)
    col = np.random.randint(0, max_pos + 1)

    # Random sprite color (RGB, normalized)
    color = np.random.rand(3) * 0.5 + 0.5  # bright colors

    for step in range(seq_length):
        # Create frame
        frame = np.zeros((grid_size, grid_size, 3), dtype=np.float32)
        frame[row:row+sprite_size, col:col+sprite_size] = color
        frames.append(frame)

        if step < seq_length - 1:
            # Choose random action
            action = np.random.randint(0, NUM_ACTIONS)
            dr, dc = ACTION_DELTAS[action]
            row = np.clip(row + dr, 0, max_pos)
            col = np.clip(col + dc, 0, max_pos)
            actions_taken.append(action)

    return np.array(frames), np.array(actions_taken)

Now let us generate the dataset and visualize some sequences.

In [ ]:
# Generate dataset
all_frames = []
all_actions = []  # kept only for evaluation, NOT given to the model

for _ in range(NUM_SEQUENCES):
    frames, actions = generate_sprite_sequence(
        SEQ_LENGTH, GRID_SIZE, SPRITE_SIZE
    )
    all_frames.append(frames)
    all_actions.append(actions)

all_frames = np.array(all_frames)   # (N, T, H, W, 3)
all_actions = np.array(all_actions) # (N, T-1) — for eval only

print(f"Dataset shape: {all_frames.shape}")
print(f"Actions shape: {all_actions.shape} (hidden from model)")
print(f"Frame value range: [{all_frames.min():.2f}, {all_frames.max():.2f}]")

### 📊 Visualization: Example Sprite Sequences

In [ ]:
fig, axes = plt.subplots(3, SEQ_LENGTH, figsize=(16, 6))
fig.suptitle("Example Sprite Sequences (Model only sees frames, NOT actions)",
             fontsize=14, fontweight='bold')

action_names = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right'}

for row in range(3):
    idx = np.random.randint(len(all_frames))
    for t in range(SEQ_LENGTH):
        axes[row, t].imshow(all_frames[idx, t])
        axes[row, t].set_xticks([])
        axes[row, t].set_yticks([])
        if t < SEQ_LENGTH - 1:
            act = action_names[all_actions[idx, t]]
            axes[row, t].set_title(f"t={t}\n({act})", fontsize=8)
        else:
            axes[row, t].set_title(f"t={t}", fontsize=8)
    axes[row, 0].set_ylabel(f"Seq {row+1}", fontsize=10)

plt.tight_layout()
plt.show()
print("Note: Action labels shown here are the GROUND TRUTH.")
print("The model will NEVER see these labels during training!")

### 4.2 Build the Video Tokenizer (VQ-VAE)

Let us build the first component: the Video Tokenizer. This is a VQ-VAE that compresses each frame into a grid of discrete tokens.

**VQ-VAE in brief:** A standard autoencoder maps inputs to a continuous latent space. A VQ-VAE adds a discrete bottleneck — the continuous encoder output is "snapped" to the nearest vector in a learnable codebook. This gives us a finite vocabulary of visual tokens.

The tricky part is the **gradient problem**: the argmin operation (finding the nearest codebook vector) is not differentiable. The solution is the **straight-through estimator** — during the backward pass, we simply copy the gradient from the decoder input to the encoder output, bypassing the quantization step.

In [ ]:
class VectorQuantizer(nn.Module):
    """Vector Quantization layer with straight-through estimator."""

    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim

        # Initialize codebook
        self.codebook = nn.Embedding(num_embeddings, embedding_dim)
        self.codebook.weight.data.uniform_(
            -1.0 / num_embeddings, 1.0 / num_embeddings
        )

    def forward(self, z):
        """Quantize continuous input to nearest codebook vectors.

        Args:
            z: (batch, height, width, embedding_dim)
        Returns:
            z_q: quantized output (same shape as z)
            indices: token indices (batch, height, width)
            vq_loss: commitment + codebook loss
        """
        # Flatten spatial dims for distance computation
        flat_z = z.reshape(-1, self.embedding_dim)  # (B*H*W, D)

        # Compute distances to all codebook entries
        distances = (
            torch.sum(flat_z ** 2, dim=1, keepdim=True)
            - 2 * flat_z @ self.codebook.weight.t()
            + torch.sum(self.codebook.weight ** 2, dim=1)
        )

        # Find nearest codebook entry
        indices = torch.argmin(distances, dim=1)  # (B*H*W,)
        z_q = self.codebook(indices).reshape(z.shape)

        # Losses
        commitment_loss = F.mse_loss(z, z_q.detach())
        codebook_loss = F.mse_loss(z.detach(), z_q)
        vq_loss = codebook_loss + 0.25 * commitment_loss

        # Straight-through estimator
        z_q = z + (z_q - z).detach()

        indices = indices.reshape(z.shape[:-1])
        return z_q, indices, vq_loss

Now let us build the encoder and decoder that wrap around the VQ layer.

In [ ]:
class Encoder(nn.Module):
    """CNN encoder: (B, 3, H, W) -> (B, h, w, D)."""

    def __init__(self, embedding_dim=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16->8
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1), # 8->4
            nn.ReLU(),
            nn.Conv2d(64, embedding_dim, 3, padding=1), # 4->4
        )

    def forward(self, x):
        # x: (B, 3, H, W) -> (B, D, h, w)
        z = self.net(x)
        # Rearrange to (B, h, w, D) for VQ
        return z.permute(0, 2, 3, 1)


class Decoder(nn.Module):
    """CNN decoder: (B, D, h, w) -> (B, 3, H, W)."""

    def __init__(self, embedding_dim=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(embedding_dim, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 3, 3, padding=1),
            nn.Sigmoid(),  # output in [0, 1]
        )

    def forward(self, z_q):
        # z_q: (B, h, w, D) -> (B, D, h, w)
        z_q = z_q.permute(0, 3, 1, 2)
        return self.net(z_q)


class VideoTokenizer(nn.Module):
    """VQ-VAE video tokenizer."""

    def __init__(self, num_codes=64, embedding_dim=32):
        super().__init__()
        self.encoder = Encoder(embedding_dim)
        self.vq = VectorQuantizer(num_codes, embedding_dim)
        self.decoder = Decoder(embedding_dim)

    def forward(self, x):
        z = self.encoder(x)
        z_q, indices, vq_loss = self.vq(z)
        x_recon = self.decoder(z_q)
        return x_recon, indices, vq_loss, z

    def encode(self, x):
        """Encode frame to token indices."""
        z = self.encoder(x)
        _, indices, _, _ = self.vq(z)
        return indices

    def decode_indices(self, indices):
        """Decode token indices back to image."""
        z_q = self.vq.codebook(indices)
        return self.decoder(z_q)

print("VideoTokenizer created.")
print(f"Each {GRID_SIZE}x{GRID_SIZE} frame -> 4x4 grid of tokens")
print(f"from a codebook of 64 entries.")

### 4.3 Build the Latent Action Model (LAM)

This is the most novel component. The LAM takes two consecutive frames (their encoded representations) and infers a discrete latent action — **without any supervision**.

The model outputs a categorical distribution over $N_a$ possible actions. During training, it uses the Gumbel-Softmax trick to allow gradients to flow through the discrete sampling.

In [ ]:
class LatentActionModel(nn.Module):
    """Infers discrete latent actions from consecutive frames."""

    def __init__(self, embedding_dim=32, spatial_size=4, num_actions=8):
        super().__init__()
        self.num_actions = num_actions
        input_dim = embedding_dim * spatial_size * spatial_size * 2

        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_actions),
        )
        self.temperature = 1.0  # Gumbel-Softmax temperature

    def forward(self, z_t, z_t1):
        """Infer latent action from two consecutive encoded frames.

        Args:
            z_t:  (B, h, w, D) — current frame encoding
            z_t1: (B, h, w, D) — next frame encoding
        Returns:
            action_onehot: (B, num_actions) — soft one-hot
            action_logits: (B, num_actions) — raw logits
        """
        batch_size = z_t.shape[0]
        z_cat = torch.cat([
            z_t.reshape(batch_size, -1),
            z_t1.reshape(batch_size, -1)
        ], dim=-1)

        logits = self.net(z_cat)

        if self.training:
            # Gumbel-Softmax for differentiable discrete sampling
            action_onehot = F.gumbel_softmax(
                logits, tau=self.temperature, hard=True
            )
        else:
            # Hard argmax at inference
            idx = logits.argmax(dim=-1)
            action_onehot = F.one_hot(
                idx, self.num_actions
            ).float()

        return action_onehot, logits

print("LatentActionModel created.")
print(f"Discovers {8} latent actions from unlabeled frame pairs.")

### 4.4 Build the Dynamics Model

The dynamics model takes the current tokenized frame and a latent action, then predicts the next tokenized frame. We use a small CNN that takes the frame embedding concatenated with a spatially-broadcast action embedding.

In [ ]:
class DynamicsModel(nn.Module):
    """Predicts next tokenized frame given current frame and action."""

    def __init__(self, embedding_dim=32, num_actions=8,
                 num_codes=64, spatial_size=4):
        super().__init__()
        self.spatial_size = spatial_size
        self.num_codes = num_codes

        # Action embedding projected to spatial map
        self.action_proj = nn.Linear(num_actions, embedding_dim)

        # Prediction network
        self.net = nn.Sequential(
            # Input: embedding_dim * 2 channels (frame + action)
            nn.Conv2d(embedding_dim * 2, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, num_codes, 1),  # predict token logits
        )

    def forward(self, z_q, action_onehot):
        """Predict next frame tokens.

        Args:
            z_q: (B, h, w, D) — current quantized frame
            action_onehot: (B, num_actions) — action one-hot
        Returns:
            token_logits: (B, num_codes, h, w)
        """
        batch_size = z_q.shape[0]
        h, w = self.spatial_size, self.spatial_size

        # Project action and broadcast to spatial grid
        action_emb = self.action_proj(action_onehot)  # (B, D)
        action_map = action_emb.unsqueeze(-1).unsqueeze(-1)
        action_map = action_map.expand(-1, -1, h, w)  # (B, D, h, w)

        # Frame features: (B, h, w, D) -> (B, D, h, w)
        frame_feat = z_q.permute(0, 3, 1, 2)

        # Concatenate and predict
        combined = torch.cat([frame_feat, action_map], dim=1)
        token_logits = self.net(combined)  # (B, num_codes, h, w)

        return token_logits

print("DynamicsModel created.")
print("Takes (frame, action) -> predicts next frame tokens.")

### 📊 Visualization: Model Architecture Overview

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 4))
ax.set_xlim(0, 14)
ax.set_ylim(0, 4)
ax.set_aspect('equal')
ax.axis('off')

# Draw components
components = [
    (1, 2, 2.0, 1.5, 'Frame $x_t$\n(16x16x3)', '#E8F5E9'),
    (3.5, 2, 2.0, 1.5, 'Encoder\n(CNN)', '#BBDEFB'),
    (6.0, 2, 2.0, 1.5, 'VQ\nCodebook', '#FFE0B2'),
    (6.0, 0.2, 2.0, 1.2, 'LAM\n(infer action)', '#F8BBD0'),
    (9.0, 2, 2.0, 1.5, 'Dynamics\nModel', '#CE93D8'),
    (11.5, 2, 2.0, 1.5, 'Next Frame\nTokens $\\hat{t}_{t+1}$', '#C8E6C9'),
]

for x, y, w, h, text, color in components:
    rect = plt.Rectangle((x, y), w, h, fill=True,
                          facecolor=color, edgecolor='black', lw=1.5)
    ax.add_patch(rect)
    ax.text(x + w/2, y + h/2, text, ha='center', va='center', fontsize=9)

# Draw arrows
arrows = [(3.0, 2.75, 0.4, 0), (5.5, 2.75, 0.4, 0),
          (8.0, 2.75, 0.9, 0), (10.9, 2.75, 0.5, 0)]
for x, y, dx, dy in arrows:
    ax.annotate('', xy=(x+dx, y+dy), xytext=(x, y),
                arrowprops=dict(arrowstyle='->', lw=2))

# Arrow from VQ to LAM
ax.annotate('', xy=(7.0, 1.4), xytext=(7.0, 2.0),
            arrowprops=dict(arrowstyle='->', lw=2, color='red'))

# Arrow from LAM to Dynamics
ax.annotate('', xy=(9.5, 1.5), xytext=(8.0, 0.8),
            arrowprops=dict(arrowstyle='->', lw=2, color='red'))
ax.text(8.5, 1.0, '$a_t$', fontsize=12, color='red', fontweight='bold')

ax.set_title("Genie Architecture: Three Components Working Together",
             fontsize=13, fontweight='bold', pad=10)
plt.tight_layout()
plt.show()

### 4.5 Prepare the Data Pipeline

In [ ]:
class SpriteDataset(Dataset):
    """Dataset of sprite movement sequences (frames only, no actions)."""

    def __init__(self, frames_array):
        # frames_array: (N, T, H, W, 3)
        # Convert to (N, T, 3, H, W) for PyTorch
        self.frames = torch.FloatTensor(frames_array).permute(0, 1, 4, 2, 3)

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        return self.frames[idx]  # (T, 3, H, W)


# Split into train/validation
n_train = int(0.9 * NUM_SEQUENCES)
train_dataset = SpriteDataset(all_frames[:n_train])
val_dataset = SpriteDataset(all_frames[n_train:])

train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_dataset, batch_size=64, shuffle=False
)

print(f"Training sequences: {len(train_dataset)}")
print(f"Validation sequences: {len(val_dataset)}")
print(f"Each sequence: {SEQ_LENGTH} frames of {GRID_SIZE}x{GRID_SIZE}")

### 4.6 Train All Three Components End-to-End

Now we train the full Genie pipeline. The training loop processes sequences of frames:

1. **Encode** all frames in the sequence with the tokenizer.
2. For each consecutive pair, **infer the latent action** with the LAM.
3. Use the dynamics model to **predict the next frame tokens**.
4. Compute the combined loss and backpropagate.

In [ ]:
# --- Hyperparameters ---
EMBEDDING_DIM = 32
NUM_CODES = 64        # codebook size
NUM_LATENT_ACTIONS = 8  # more than true 4, model must discover
LEARNING_RATE = 3e-4
NUM_EPOCHS = 30
BETA_VQ = 1.0         # weight for VQ loss

# --- Initialize models ---
tokenizer = VideoTokenizer(NUM_CODES, EMBEDDING_DIM).to(device)
lam = LatentActionModel(
    EMBEDDING_DIM, spatial_size=4, num_actions=NUM_LATENT_ACTIONS
).to(device)
dynamics = DynamicsModel(
    EMBEDDING_DIM, NUM_LATENT_ACTIONS, NUM_CODES, spatial_size=4
).to(device)

# --- Single optimizer for all components ---
all_params = (
    list(tokenizer.parameters())
    + list(lam.parameters())
    + list(dynamics.parameters())
)
optimizer = torch.optim.Adam(all_params, lr=LEARNING_RATE)

print(f"Tokenizer params:  {sum(p.numel() for p in tokenizer.parameters()):,}")
print(f"LAM params:        {sum(p.numel() for p in lam.parameters()):,}")
print(f"Dynamics params:   {sum(p.numel() for p in dynamics.parameters()):,}")
print(f"Total params:      {sum(p.numel() for p in all_params):,}")

In [ ]:
# --- Training Loop ---
history = {
    'recon_loss': [], 'vq_loss': [],
    'dynamics_loss': [], 'total_loss': []
}

for epoch in range(NUM_EPOCHS):
    tokenizer.train()
    lam.train()
    dynamics.train()

    epoch_losses = {k: 0.0 for k in history}
    n_batches = 0

    for batch_sequences in train_loader:
        batch_sequences = batch_sequences.to(device)  # (B, T, 3, H, W)
        B, T = batch_sequences.shape[:2]

        total_recon = 0.0
        total_vq = 0.0
        total_dyn = 0.0

        # Encode all frames
        all_z = []
        all_z_q = []
        all_indices = []

        for t in range(T):
            frame = batch_sequences[:, t]          # (B, 3, H, W)
            recon, indices, vq_loss, z = tokenizer(frame)
            z_q = tokenizer.vq.codebook(indices)   # (B, h, w, D)

            total_recon += F.mse_loss(recon, frame)
            total_vq += vq_loss
            all_z.append(z)
            all_z_q.append(z_q)
            all_indices.append(indices)

        # For consecutive pairs, infer actions and predict next frame
        for t in range(T - 1):
            # Infer latent action
            action_onehot, _ = lam(all_z[t], all_z[t+1])

            # Predict next frame tokens
            token_logits = dynamics(all_z_q[t], action_onehot)
            target_indices = all_indices[t+1]  # (B, h, w)

            # Cross-entropy loss over token predictions
            dyn_loss = F.cross_entropy(
                token_logits, target_indices
            )
            total_dyn += dyn_loss

        # Average losses over time steps
        loss_recon = total_recon / T
        loss_vq = total_vq / T
        loss_dyn = total_dyn / (T - 1)

        loss = loss_recon + BETA_VQ * loss_vq + loss_dyn

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(all_params, 1.0)
        optimizer.step()

        epoch_losses['recon_loss'] += loss_recon.item()
        epoch_losses['vq_loss'] += loss_vq.item()
        epoch_losses['dynamics_loss'] += loss_dyn.item()
        epoch_losses['total_loss'] += loss.item()
        n_batches += 1

    # Record averages
    for k in history:
        avg = epoch_losses[k] / n_batches
        history[k].append(avg)

    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{NUM_EPOCHS} | "
              f"Recon: {history['recon_loss'][-1]:.4f} | "
              f"VQ: {history['vq_loss'][-1]:.4f} | "
              f"Dyn: {history['dynamics_loss'][-1]:.4f} | "
              f"Total: {history['total_loss'][-1]:.4f}")

print("\nTraining complete!")

### 📊 Visualization: Tokenizer Reconstruction Quality

Let us see how well our tokenizer reconstructs frames. This tells us whether the discrete bottleneck preserves enough information.

In [ ]:
tokenizer.eval()
with torch.no_grad():
    sample_batch = next(iter(val_loader)).to(device)
    sample_frames = sample_batch[:8, 0]  # 8 frames from first timestep
    recon, _, _, _ = tokenizer(sample_frames)

fig, axes = plt.subplots(2, 8, figsize=(16, 4))
fig.suptitle("Video Tokenizer: Original (top) vs Reconstructed (bottom)",
             fontsize=13, fontweight='bold')

for i in range(8):
    orig = sample_frames[i].cpu().permute(1, 2, 0).numpy()
    rec = recon[i].cpu().permute(1, 2, 0).numpy()

    axes[0, i].imshow(orig)
    axes[0, i].set_title(f"Original", fontsize=8)
    axes[0, i].axis('off')

    axes[1, i].imshow(rec)
    axes[1, i].set_title(f"Reconstructed", fontsize=8)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

### 📊 Visualization: Discovered Latent Actions

Now the exciting part — let us see whether the Latent Action Model has discovered meaningful actions. We will color-code frame transitions by their inferred latent action and see if they cluster by true action.

In [ ]:
tokenizer.eval()
lam.eval()

# Collect inferred vs true actions
inferred_actions = []
true_actions_list = []

with torch.no_grad():
    for seq_idx in range(min(500, n_train)):
        frames = train_dataset[seq_idx].unsqueeze(0).to(device)  # (1,T,3,H,W)

        for t in range(SEQ_LENGTH - 1):
            frame_t = frames[:, t]
            frame_t1 = frames[:, t+1]

            z_t = tokenizer.encoder(frame_t)
            z_t1 = tokenizer.encoder(frame_t1)

            action_onehot, logits = lam(z_t, z_t1)
            inferred = logits.argmax(dim=-1).item()

            inferred_actions.append(inferred)
            true_actions_list.append(all_actions[seq_idx, t])

inferred_actions = np.array(inferred_actions)
true_actions_list = np.array(true_actions_list)

# Build confusion-style matrix
action_names_true = ['Up', 'Down', 'Left', 'Right']
confusion = np.zeros((NUM_LATENT_ACTIONS, NUM_ACTIONS))
for inf_a, true_a in zip(inferred_actions, true_actions_list):
    confusion[inf_a, true_a] += 1

# Normalize rows
row_sums = confusion.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
confusion_norm = confusion / row_sums

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix
im = axes[0].imshow(confusion_norm, cmap='Blues', aspect='auto')
axes[0].set_xlabel("True Action")
axes[0].set_ylabel("Inferred Latent Action")
axes[0].set_xticks(range(NUM_ACTIONS))
axes[0].set_xticklabels(action_names_true)
axes[0].set_yticks(range(NUM_LATENT_ACTIONS))
axes[0].set_title("Latent Action vs True Action\n(each row should map to one true action)")
plt.colorbar(im, ax=axes[0])

# Action distribution
action_counts = np.bincount(inferred_actions, minlength=NUM_LATENT_ACTIONS)
axes[1].bar(range(NUM_LATENT_ACTIONS), action_counts, color='steelblue')
axes[1].set_xlabel("Latent Action Index")
axes[1].set_ylabel("Count")
axes[1].set_title("Distribution of Discovered Actions")

plt.tight_layout()
plt.show()

# Compute cluster purity
purity = 0
for i in range(NUM_LATENT_ACTIONS):
    if confusion[i].sum() > 0:
        purity += confusion[i].max()
purity /= confusion.sum()
print(f"Cluster purity: {purity:.2%}")
print("(Higher = latent actions better align with true actions)")

### 📊 Visualization: Dynamics Model — Predicted vs Actual Next Frame

In [ ]:
tokenizer.eval()
lam.eval()
dynamics.eval()

fig, axes = plt.subplots(4, 3, figsize=(9, 12))
col_titles = ['Current Frame', 'Actual Next', 'Predicted Next']

with torch.no_grad():
    for row in range(4):
        idx = np.random.randint(len(val_dataset))
        t = np.random.randint(SEQ_LENGTH - 1)

        frames = val_dataset[idx].unsqueeze(0).to(device)
        frame_t = frames[:, t]
        frame_t1 = frames[:, t + 1]

        # Encode and infer action
        z_t = tokenizer.encoder(frame_t)
        z_t1 = tokenizer.encoder(frame_t1)
        z_q_t, indices_t, _, _ = tokenizer.vq(z_t)

        action_onehot, _ = lam(z_t, z_t1)

        # Predict next frame tokens
        token_logits = dynamics(z_q_t, action_onehot)
        pred_indices = token_logits.argmax(dim=1)  # (B, h, w)

        # Decode both actual and predicted
        actual_recon = tokenizer.decode_indices(
            tokenizer.encode(frame_t1)
        )
        pred_recon = tokenizer.decode_indices(pred_indices)

        # Display
        for col, img_tensor in enumerate([frame_t, frame_t1, pred_recon]):
            img = img_tensor[0].cpu().permute(1, 2, 0).numpy()
            img = np.clip(img, 0, 1)
            axes[row, col].imshow(img)
            axes[row, col].axis('off')
            if row == 0:
                axes[row, col].set_title(col_titles[col], fontsize=11)

fig.suptitle("Dynamics Model: Current → Predicted vs Actual Next Frame",
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. 🔧 Your Turn (TODO)

Now it is your turn! We have two exercises that test your understanding of the core mechanisms.

### TODO 1: Implement Vector Quantization

The vector quantization step is the heart of the tokenizer. Given a continuous encoder output, you must:
1. Compute distances to all codebook entries
2. Find the nearest codebook vector
3. Apply the straight-through estimator so gradients flow through

In [ ]:
def vector_quantize(z, codebook_weight):
    """Implement the vector quantization step.

    Args:
        z: Continuous encoder output (batch, height, width, embedding_dim)
        codebook_weight: Codebook vectors (num_codes, embedding_dim)

    Returns:
        z_q: Quantized output (same shape as z)
        indices: Token indices (batch, height, width)

    Hints:
        - Flatten z to (B*H*W, D) for distance computation
        - Use the squared distance formula: ||a-b||^2 = ||a||^2 - 2*a.b + ||b||^2
        - For straight-through: z_q_st = z + (z_q - z).detach()
    """
    B, H, W, D = z.shape

    # ============ TODO ============
    # Step 1: Flatten z to (B*H*W, D)
    # Step 2: Compute squared distances to each codebook entry
    # Step 3: Find nearest codebook indices (argmin)
    # Step 4: Look up codebook vectors for those indices
    # Step 5: Reshape z_q back to (B, H, W, D)
    # Step 6: Apply straight-through estimator
    # ==============================

    flat_z = ???         # YOUR CODE: flatten
    distances = ???      # YOUR CODE: squared distances
    indices = ???        # YOUR CODE: argmin
    z_q = ???            # YOUR CODE: look up codebook vectors
    z_q = z_q.reshape(B, H, W, D)
    z_q_st = ???         # YOUR CODE: straight-through estimator

    indices = indices.reshape(B, H, W)
    return z_q_st, indices

### Verification Cell for TODO 1

In [ ]:
# ✅ Verification: Run this cell to check your implementation
torch.manual_seed(42)
test_z = torch.randn(2, 4, 4, 32)
test_codebook = torch.randn(64, 32)

try:
    test_zq, test_indices = vector_quantize(test_z, test_codebook)

    assert test_zq.shape == test_z.shape, \
        f"Wrong output shape: expected {test_z.shape}, got {test_zq.shape}"
    assert test_indices.shape == (2, 4, 4), \
        f"Wrong indices shape: expected (2, 4, 4), got {test_indices.shape}"
    assert test_indices.dtype == torch.int64, \
        f"Indices should be int64, got {test_indices.dtype}"
    assert torch.all(test_indices >= 0) and torch.all(test_indices < 64), \
        "Indices should be in range [0, 64)"

    # Check straight-through: z_q should require grad if z does
    test_z_grad = test_z.clone().requires_grad_(True)
    test_zq_grad, _ = vector_quantize(test_z_grad, test_codebook)
    loss = test_zq_grad.sum()
    loss.backward()
    assert test_z_grad.grad is not None, \
        "Gradients should flow through straight-through estimator"

    print("✅ Correct! Your vector quantization implementation works perfectly.")
    print(f"   Output shape: {test_zq.shape}")
    print(f"   Indices range: [{test_indices.min().item()}, {test_indices.max().item()}]")
    print(f"   Gradients flow: True")
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Check your implementation and try again.")

### TODO 2: Implement the Interactive Generation Loop

This is where everything comes together for interactivity. Given a starting frame and a sequence of user-chosen actions, generate a trajectory of frames — just like playing a game!

In [ ]:
@torch.no_grad()
def interactive_generate(start_frame, action_sequence,
                         tokenizer, lam, dynamics):
    """Generate a sequence of frames interactively.

    Args:
        start_frame: Initial frame tensor (1, 3, H, W)
        action_sequence: List of latent action indices [int]
        tokenizer: Trained VideoTokenizer
        lam: Trained LatentActionModel
        dynamics: Trained DynamicsModel

    Returns:
        generated_frames: List of numpy images (H, W, 3)

    Hints:
        - Encode the starting frame with the tokenizer
        - For each action in the sequence:
            1. Convert action index to one-hot
            2. Feed (current_z_q, action_onehot) to dynamics model
            3. Get predicted token indices from logits (argmax)
            4. Decode predicted indices to an image
            5. Re-encode the decoded image for the next step
        - Collect each generated frame as a numpy image
    """
    tokenizer.eval()
    dynamics.eval()

    generated_frames = []

    # Store the starting frame
    start_img = start_frame[0].cpu().permute(1, 2, 0).numpy()
    generated_frames.append(np.clip(start_img, 0, 1))

    # ============ TODO ============
    # Step 1: Encode starting frame -> z, z_q, indices
    # Step 2: Loop through action_sequence:
    #     a. Create one-hot action vector
    #     b. Predict next token logits with dynamics model
    #     c. Get predicted indices (argmax over dim=1)
    #     d. Decode indices to image with tokenizer.decode_indices
    #     e. Append decoded image (as numpy) to generated_frames
    #     f. Re-encode decoded image for next iteration
    # ==============================

    # YOUR CODE HERE

    return generated_frames

### Verification Cell for TODO 2

In [ ]:
# ✅ Verification: Run this cell to check your implementation
tokenizer.eval()
dynamics.eval()

# Get a starting frame
test_seq = val_dataset[0].unsqueeze(0).to(device)
test_start = test_seq[:, 0]  # (1, 3, H, W)
test_actions = [0, 1, 2, 3, 0, 1]  # 6-step trajectory

try:
    generated = interactive_generate(
        test_start, test_actions, tokenizer, lam, dynamics
    )

    assert len(generated) == len(test_actions) + 1, \
        f"Expected {len(test_actions)+1} frames, got {len(generated)}"
    assert generated[0].shape == (GRID_SIZE, GRID_SIZE, 3), \
        f"Wrong frame shape: expected ({GRID_SIZE},{GRID_SIZE},3), got {generated[0].shape}"

    # Visualize the generated trajectory
    fig, axes = plt.subplots(1, len(generated), figsize=(16, 3))
    fig.suptitle("✅ Interactive Generation — Your Implementation Works!",
                 fontsize=13, fontweight='bold')
    for i, frame in enumerate(generated):
        axes[i].imshow(np.clip(frame, 0, 1))
        axes[i].axis('off')
        if i == 0:
            axes[i].set_title("Start", fontsize=9)
        else:
            axes[i].set_title(f"a={test_actions[i-1]}", fontsize=9)
    plt.tight_layout()
    plt.show()

    print("✅ Correct! Your interactive generation loop works perfectly.")
    print(f"   Generated {len(generated)} frames from {len(test_actions)} actions.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Check your implementation and try again.")

## 6. Putting It All Together — The Full Genie Pipeline

Now let us step back and see the complete pipeline working end-to-end. This function runs the full Genie process: tokenize, infer actions, predict dynamics.

In [ ]:
@torch.no_grad()
def full_genie_pipeline(sequence, tokenizer, lam, dynamics):
    """Run the full Genie pipeline on a sequence of frames.

    Args:
        sequence: (T, 3, H, W) tensor of frames

    Returns:
        dict with all intermediate outputs
    """
    tokenizer.eval()
    lam.eval()
    dynamics.eval()

    T = sequence.shape[0]
    sequence = sequence.unsqueeze(0).to(device)  # (1, T, 3, H, W)

    results = {
        'originals': [], 'reconstructed': [],
        'token_indices': [], 'inferred_actions': [],
        'predicted_next': []
    }

    # Stage 1: Tokenize all frames
    encoded_z = []
    for t in range(T):
        frame = sequence[:, t]
        recon, indices, _, z = tokenizer(frame)
        z_q = tokenizer.vq.codebook(indices)

        results['originals'].append(
            frame[0].cpu().permute(1, 2, 0).numpy()
        )
        results['reconstructed'].append(
            recon[0].cpu().permute(1, 2, 0).numpy()
        )
        results['token_indices'].append(indices[0].cpu().numpy())
        encoded_z.append((z, z_q, indices))

    # Stage 2: Infer actions between consecutive frames
    for t in range(T - 1):
        z_t = encoded_z[t][0]
        z_t1 = encoded_z[t + 1][0]
        action_onehot, logits = lam(z_t, z_t1)
        action_idx = logits.argmax(dim=-1).item()
        results['inferred_actions'].append(action_idx)

        # Stage 3: Predict next frame
        z_q_t = encoded_z[t][1]
        token_logits = dynamics(z_q_t, action_onehot)
        pred_indices = token_logits.argmax(dim=1)
        pred_frame = tokenizer.decode_indices(pred_indices)
        results['predicted_next'].append(
            pred_frame[0].cpu().permute(1, 2, 0).numpy()
        )

    return results

# Run on a validation sequence
test_sequence = val_dataset[42]
results = full_genie_pipeline(test_sequence, tokenizer, lam, dynamics)
print("Full Genie pipeline complete!")
print(f"Processed {len(results['originals'])} frames")
print(f"Inferred {len(results['inferred_actions'])} actions")

## 7. Training and Results

### 📊 Training Loss Curves

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

loss_names = ['recon_loss', 'vq_loss', 'dynamics_loss', 'total_loss']
titles = [
    'Reconstruction Loss\n(Tokenizer quality)',
    'VQ Commitment Loss\n(Codebook alignment)',
    'Dynamics Loss\n(Next-frame prediction)',
    'Total Loss'
]
colors = ['#2196F3', '#FF9800', '#4CAF50', '#9C27B0']

for ax, name, title, color in zip(axes, loss_names, titles, colors):
    ax.plot(history[name], color=color, linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title(title, fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, NUM_EPOCHS - 1)

fig.suptitle("Training Progress — All Three Components",
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Action Discovery Quality

How well did the model discover the true 4 actions from unlabeled data? Let us compute a more detailed analysis.

In [ ]:
# Re-compute with more data
tokenizer.eval()
lam.eval()

all_inf = []
all_true = []

with torch.no_grad():
    for seq_idx in range(min(1000, n_train)):
        frames = train_dataset[seq_idx].unsqueeze(0).to(device)
        for t in range(SEQ_LENGTH - 1):
            z_t = tokenizer.encoder(frames[:, t])
            z_t1 = tokenizer.encoder(frames[:, t+1])
            _, logits = lam(z_t, z_t1)
            all_inf.append(logits.argmax(dim=-1).item())
            all_true.append(all_actions[seq_idx, t])

all_inf = np.array(all_inf)
all_true = np.array(all_true)

# For each latent action, find its dominant true action
latent_to_true = {}
for la in range(NUM_LATENT_ACTIONS):
    mask = all_inf == la
    if mask.sum() > 0:
        counts = np.bincount(all_true[mask], minlength=NUM_ACTIONS)
        dominant = counts.argmax()
        latent_to_true[la] = {
            'dominant': action_names_true[dominant],
            'purity': counts.max() / counts.sum(),
            'count': mask.sum()
        }

print("Latent Action Discovery Results:")
print("-" * 50)
print(f"{'Latent':>8} {'Maps to':>10} {'Purity':>10} {'Count':>8}")
print("-" * 50)
for la in sorted(latent_to_true.keys()):
    info = latent_to_true[la]
    print(f"{la:>8} {info['dominant']:>10} {info['purity']:>9.1%} {info['count']:>8}")

# Overall accuracy (with best mapping)
mapped_correct = sum(
    1 for i, t in zip(all_inf, all_true)
    if i in latent_to_true and
    latent_to_true[i]['dominant'] == action_names_true[t]
)
overall_acc = mapped_correct / len(all_true)
print(f"\nOverall action discovery accuracy: {overall_acc:.1%}")

## 8. 🎯 Final Output — Interactive World Generation

This is the moment everything comes together. We will generate interactive worlds by starting from a single frame and applying different action sequences. **Same starting frame, different action sequences lead to different worlds.**

In [ ]:
@torch.no_grad()
def generate_world(start_frame, action_sequence,
                   tokenizer, dynamics, device):
    """Generate an interactive world from a starting frame.

    This is the core Genie inference loop.
    """
    tokenizer.eval()
    dynamics.eval()

    frames = [start_frame[0].cpu().permute(1, 2, 0).numpy()]

    # Encode starting frame
    z = tokenizer.encoder(start_frame)
    z_q, indices, _, _ = tokenizer.vq(z)

    for action_idx in action_sequence:
        # Create action one-hot
        action_oh = F.one_hot(
            torch.tensor([action_idx]), NUM_LATENT_ACTIONS
        ).float().to(device)

        # Predict next frame tokens
        token_logits = dynamics(z_q, action_oh)
        pred_indices = token_logits.argmax(dim=1)

        # Decode to image
        pred_frame = tokenizer.decode_indices(pred_indices)
        frames.append(
            pred_frame[0].cpu().permute(1, 2, 0).numpy()
        )

        # Re-encode for next step
        z = tokenizer.encoder(pred_frame)
        z_q, _, _, _ = tokenizer.vq(z)

    return [np.clip(f, 0, 1) for f in frames]

Now let us generate worlds with different action sequences from the same starting frame.

In [ ]:
# Find the most-used latent action for each true direction
best_actions = {}
for la, info in latent_to_true.items():
    direction = info['dominant']
    if direction not in best_actions or info['count'] > best_actions[direction][1]:
        best_actions[direction] = (la, info['count'])

print("Best latent action mapping:")
for direction, (la, count) in best_actions.items():
    print(f"  {direction}: latent action {la} ({count} examples)")

In [ ]:
# Generate a starting frame
np.random.seed(99)
start_frames_np, _ = generate_sprite_sequence(
    1, GRID_SIZE, SPRITE_SIZE
)
start_tensor = torch.FloatTensor(
    start_frames_np[0]
).permute(2, 0, 1).unsqueeze(0).to(device)

# Define 4 different "journeys" from the same starting point
# Use the discovered latent actions
journey_length = 10
journeys = {}

# Try to use discovered mappings, fallback to simple indices
action_map = {d: la for d, (la, _) in best_actions.items()}
up_a = action_map.get('Up', 0)
down_a = action_map.get('Down', 1)
left_a = action_map.get('Left', 2)
right_a = action_map.get('Right', 3)

journeys['Go Right'] = [right_a] * journey_length
journeys['Go Down'] = [down_a] * journey_length
journeys['Zigzag (R/D)'] = [right_a, down_a] * (journey_length // 2)
journeys['Spiral'] = (
    [right_a]*3 + [down_a]*2 + [left_a]*3 + [up_a]*2
)

fig, axes = plt.subplots(
    len(journeys), journey_length + 1, figsize=(20, 8)
)
fig.suptitle(
    "🎯 Interactive World Generation\n"
    "Same Starting Frame → Different Action Sequences → Different Worlds",
    fontsize=14, fontweight='bold'
)

for row, (journey_name, action_seq) in enumerate(journeys.items()):
    frames = generate_world(
        start_tensor, action_seq, tokenizer, dynamics, device
    )

    for col in range(min(len(frames), journey_length + 1)):
        axes[row, col].imshow(frames[col])
        axes[row, col].axis('off')
        if col == 0:
            axes[row, col].set_title("Start", fontsize=8)
        elif col < len(action_seq) + 1:
            axes[row, col].set_title(f"t={col}", fontsize=8)

    axes[row, 0].set_ylabel(journey_name, fontsize=11, rotation=0,
                            labelpad=70, va='center')

plt.tight_layout()
plt.show()

### The Grand Finale: Random Exploration

Let us generate a long random trajectory — the model exploring its own generated world.

In [ ]:
# Long random exploration
np.random.seed(7)
random_actions = np.random.choice(
    list(range(NUM_LATENT_ACTIONS)), size=20
).tolist()

exploration_frames = generate_world(
    start_tensor, random_actions, tokenizer, dynamics, device
)

# Display as a 3-row grid
n_cols = 7
n_rows = 3
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 7))
fig.suptitle(
    "Random Exploration: 20 Steps of Autonomous World Generation",
    fontsize=14, fontweight='bold'
)

for i in range(min(n_rows * n_cols, len(exploration_frames))):
    r, c = i // n_cols, i % n_cols
    axes[r, c].imshow(exploration_frames[i])
    axes[r, c].axis('off')
    axes[r, c].set_title(f"t={i}", fontsize=9)

# Turn off unused axes
for i in range(len(exploration_frames), n_rows * n_cols):
    r, c = i // n_cols, i % n_cols
    axes[r, c].axis('off')

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("🎉 Congratulations!")
print("=" * 60)
print()
print("You have built a simplified Genie-style model that:")
print("  1. Compresses frames into discrete tokens (VQ-VAE)")
print("  2. Discovers actions from unlabeled video (LAM)")
print("  3. Predicts interactive next frames (Dynamics Model)")
print()
print("From a SINGLE starting frame and a sequence of actions,")
print("you can now generate entire interactive trajectories —")
print("just like playing a video game that was never programmed!")
print("=" * 60)

## 9. Reflection and Next Steps

### 🤔 Reflection Questions

1. **Why discrete actions?** Our Latent Action Model outputs a *discrete* categorical distribution over actions, not a continuous vector. Why is discreteness important for interactive control? What would happen if we used a continuous latent action space instead?

2. **Codebook size matters.** We used a codebook of 64 entries for the video tokenizer. What would happen if we used only 4 entries? What about 1024? How does codebook size relate to the reconstruction quality vs. compactness tradeoff?

3. **Scaling to real images.** Our sprites are 16x16 pixels. Genie 1 works on 256x256 platformer games. Genie 2 generates minute-long videos. What architectural changes would be needed to scale this approach to realistic images? (Hint: think about the tokenizer, the dynamics model architecture, and training data.)

### 🔧 Optional Challenges

**Challenge 1: Better Dynamics Model.** Replace the CNN dynamics model with a small Transformer. Instead of operating on the 2D spatial grid with convolutions, flatten the token grid into a sequence, prepend the action token, and use self-attention to predict next-frame tokens. Does this improve prediction quality?

**Challenge 2: Conditional Starting Frames.** Instead of using a frame from the dataset, try creating a hand-drawn starting frame (e.g., draw a white square at a specific position). Feed it to the model and generate an interactive trajectory. Does the model generalize to starting frames it has never seen during training? This is exactly what Genie does — it takes a *sketch* and makes it playable.

### Genie at Scale: From Genie 1 to Genie 3

Our simplified version captures the core idea, but the real Genie models are far more powerful:

| Feature | Our Notebook | Genie 1 (2024) | Genie 2 (2024) | Genie 3 (2025) |
|---------|-------------|-----------------|-----------------|-----------------|
| **Resolution** | 16x16 | 256x256 | 720p | 1080p |
| **Duration** | 8 frames | 16 frames | ~60 seconds | Real-time |
| **Training data** | 3K sprite seqs | 200K hrs video | Internet-scale | Internet-scale |
| **Tokenizer** | Small CNN VQ-VAE | ST-ViViT | Large VQ-VAE | Advanced VQ |
| **Dynamics** | Small CNN | 11B param Transformer | Large Transformer | Optimized Transformer |
| **Actions** | 4 discovered | 8 discovered | Rich action space | Continuous control |
| **3D** | No | No | Limited | Full 3D worlds |

The progression from Genie 1 to Genie 3 shows a clear trajectory: **from simple 2D platformers to real-time, interactive 3D worlds** — all learned from unlabeled video.

This brings us to a profound question that we will explore in the next notebook: If a model can generate interactive worlds from a single image, what happens when we give it the ability to *act* in those worlds? This is where world models meet embodied agents — and where the future of AI truly begins.

### References

- **Genie 1:** Bruce et al., "Genie: Generative Interactive Environments," *ICML 2024*. [arXiv:2402.15391](https://arxiv.org/abs/2402.15391)
- **Genie 2:** DeepMind, "Genie 2: A Large-Scale Foundation World Model," *2024*. [Blog post](https://deepmind.google/discover/blog/genie-2-a-large-scale-foundation-world-model/)
- **VQ-VAE:** van den Oord et al., "Neural Discrete Representation Learning," *NeurIPS 2017*. [arXiv:1711.00937](https://arxiv.org/abs/1711.00937)

See you in the next notebook!